In [ ]:
import numpy as np
import math


We will use the Bush-Mosteller approach:
![probability for cooperating for public goods game](https://journals.plos.org/ploscompbiol/article/file?type=thumbnail&id=info:doi/10.1371/journal.pcbi.1005034.e021)

![the stimulus (s) is based on whether or not the player’s actual payoff met some aspiration (expectation)](https://journals.plos.org/ploscompbiol/article/file?type=thumbnail&id=info:doi/10.1371/journal.pcbi.1005034.e002)



* basic idea behind this is that whether or not a person contributes is based on whether or not they think the group’s work met an expectation of theirs
* the initial condition p<sub>1</sub> is from the uniform density on [0, 1] independently for different players.
* First, we define p<sub>t</sub> as the expected contribution that the player makes in round t. We draw the actual contribution at from the truncated Gaussian distribution whose mean and standard deviation are equal to pt and 0.2, respectively. If a<sub>t</sub> falls outside the interval [0, 1], we discard it and redraw at until it falls within [0, 1]. Second, we introduce a threshold contribution value X, distinct from A, used for regarding the action to be either cooperative or defective.
    * X = 0.3 and 0.4 for conditional cooperative behavior
    * beta = 0.4, A = 0.9

In [ ]:
iteNum = 1000
N = 4 # number of agents
tmax = 25 # number of rounds

a = 1.6 # multiply factor for PGG

std = 0.2 # std of distribution from which contribution is drawn
beta = 0.4
A = 1.0
X = 0.5 # cooperativeness criteria

In [ ]:
def initialize(net, payoff, at, pt, st, At):
    for i in range(N):
        payoff[i] = 0
        pt[i] = np.random.rand()
        at[i] = np.random.normal() * std + pt[i] # initial contribution

        while at[i] < 0 or at[i] > 1:
            at[i] = np.random.normal() * std + pt[i] # discard irrational contribution
        
        At[i] = A
        st[i] = 0

def completeNet():
    net = [[] for i in range(N)]

    for i in range(N):
        for j in range(N):
            if i != j:
                net[i].append(j)
    
    return net

In [ ]:
def updatePGG(net, payoff, at, pt, st, At):
    # compute payoffs

    for i in range(N):
        pool = 0
        for j in range(len(net[i])): # collect contributions from neighbors
            pool += at[net[i][j]] * a
        
        pool += at[i] * a
        payoff[i] = 1 - at[i] + pool / (len(net[i]) + 1)
    
    # update pt[]
    for i in range(N):
        st[i] = math.tanh(beta * (payoff[i] - At[i]))

        if (at[i] >= X): # cooperation
            if st[i] >= 0:
                pt[i] = pt[i] + (1 - pt[i]) * st[i]
            else:
                pt[i] = pt[i] + pt[i] * st[i]
        else: # defected
            if st[i] >= 0:
                pt[i] = pt[i] - pt[i] * st[i]
            else:
                pt[i] = pt[i] - (1 - pt[i]) * st[i]
    
    # draw contribution for next round
    for i in range(N):
        at[i] = np.random.normal() * std + pt[i]
        while at[i] < 0 or at[i] > 1:
            at[i] = np.random.normal() * std + pt[i]

In [ ]:
def main():
    aveCont = [0.0] * tmax
    net = completeNet()
    pt = [0.0] * N
    At = [0.0] * N
    st = [0.0] * N
    at = [0.0] * N
    payoff = [0.0] * N

    for ite in range(iteNum):
        initialize(net, payoff, at, pt, st, At)
        for t in range(tmax):
            updatePGG(net, payoff, at, pt, st, At)
            aveCont[t] += np.mean(at)
    
    print("Round: Average Contribution")
    for t in range(tmax):
        print('{0}: {1:0.3f}'.format(t + 1, aveCont[t] / iteNum))

main()

Round: Average Contribution
1: 0.482
2: 0.471
3: 0.459
4: 0.448
5: 0.438
6: 0.429
7: 0.422
8: 0.417
9: 0.405
10: 0.397
11: 0.395
12: 0.385
13: 0.378
14: 0.366
15: 0.367
16: 0.359
17: 0.356
18: 0.347
19: 0.346
20: 0.340
21: 0.337
22: 0.330
23: 0.325
24: 0.327
25: 0.323
